In [1]:
# import modules
"""
Created on Tue Feb 20 15:10:00 2018

@author: Yacalis
"""

import os
import json
import numpy as np
from Callbacks import Callbacks
from Config import Config
from DataLoader import retrieve_data
from folder_defs import get_log_dir, get_data_dir, get_train_dir, get_test_dir
from train_model import train_model
from build_model import build_model
from save_model import save_model
from get_data_dict import get_data_dict

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
print('Beginning program')

Beginning program


In [3]:
# get config
config = Config().config
print('change lr:', config.change_lr)
print('change bs:', config.change_bs)
print('max epochs:', config.epochs)
if config.change_bs == config.change_lr:
    print(f'[!] Whoops: config.change_bs and config.change_lr should be '
          f'different bool values, but they are both {config.change_bs} '
          f'-- please set one and only one of them to True')

unparsed config options: ['-f', '/Users/Yacalis/Library/Jupyter/runtime/kernel-4b96d059-124e-41da-ad50-f2f0fb96722c.json']
change lr: True
change bs: False
max epochs: 20


In [4]:
# get directories
log_dir = get_log_dir(config)
data_dir = get_data_dir()
train_dir = get_train_dir()
test_dir = get_test_dir()
print('log dir:', log_dir)
print('data dir:', data_dir)
print('train dir:', train_dir)
print('test dir:', test_dir)

log dir: /Users/Yacalis/Projects/TensorFlow/cs274c-data/logs/0306_151300-changelr_True-changebs_False-batch_4
data dir: /Users/Yacalis/Projects/TensorFlow/cs274c-data/
train dir: /Users/Yacalis/Projects/TensorFlow/cs274c-data/test_images/train/
test dir: /Users/Yacalis/Projects/TensorFlow/cs274c-data/test_images/val/


In [5]:
# get data
print('Loading data...')
data_dict = get_data_dict(data_dir)
x_data, y_data = retrieve_data(data_dict=data_dict, image_dir=train_dir)
num_train = int(x_data.shape[0] * 0.8)
print(f'Num training examples (excludes test and val): {num_train}')

Loading data...
data file:  /Users/Yacalis/Projects/TensorFlow/cs274c-data/imdb.mat
number of records from data file:  460723
image dir:  /Users/Yacalis/Projects/TensorFlow/cs274c-data/test_images/train/
	sub dirs:
		 00
		 01
		 02
		 03
		 04
		 05
shape of x_data:  (4229, 228, 228, 3)
Num training examples (excludes test and val): 3383


In [6]:
# build and save initial model
input_dim = x_data[0].shape
model = build_model(input_dim, config)
save_model(log_dir=log_dir, config=config, model=model)

Building model...
Finished building model
Compiling model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 56, 56, 96)        14208     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 256)       614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 11, 11, 256)       1024      
_________________________________________________________________
conv2d_3 (Conv2

In [7]:
# set variables
val_loss = []
val_acc = []
loss = []
acc = []
lr = []
bs = []
max_epochs = config.epochs
batch_size = config.batch_size
batch_size_mult = 2
epoch_iter = 1

In [8]:
# get callbacks
callbacks = Callbacks(config, log_dir).callbacks
print('callbacks:')
for callback in callbacks:
    print('\t', callback)

callbacks:


In [ ]:
# train model
if config.change_lr:  # reduce_lr callback takes care of everything for us
    print('Will reduce learning rate during training, but not batch size')
    print('Training model...')
    model, history = train_model(model, x_data, y_data, batch_size, max_epochs, callbacks)

    # store history (bs is constant)
    val_loss += history.history['val_loss']
    val_acc += history.history['val_acc']
    loss += history.history['loss']
    acc += history.history['acc']
    lr += history.history['lr']
    bs = [batch_size for i in range(len(lr))]

elif config.change_bs:  # need to manually stop and restart training
    print('Will reduce batch size during training, but not learning rate')
    while max_epochs >= epoch_iter:
        print(f'Currently at epoch {epoch_iter} of {max_epochs}, batch size is {batch_size}')
        epochs = max_epochs - epoch_iter + 1
        model, history = train_model(model, x_data, y_data, batch_size, epochs, callbacks)

        # store history
        val_loss += history.history['val_loss']
        val_acc += history.history['val_acc']
        loss += history.history['loss']
        acc += history.history['acc']
        bs += [batch_size for i in range(len(history.epoch))]

        # update training parameters
        epoch_iter += len(history.epoch)
        batch_size *= batch_size_mult
        batch_size = batch_size if batch_size < num_train else num_train

    # store lr history as constant
    lr = [0.001 for i in range(len(bs))]

else:  # this should never happen
    print(f'[!] Whoops: config.change_bs and config.change_lr are both '
          f'set to False - please set one of them to True')

print('Completed training')

Will reduce learning rate during training, but not batch size
Training model...
Train on 3383 samples, validate on 846 samples
Epoch 1/20
3383/3383 [==============================] - 369s 109ms/step - loss: 6.6812 - acc: 0.5723 - val_loss: 7.0874 - val_acc: 0.5603
Epoch 2/20
3383/3383 [==============================] - 379s 112ms/step - loss: 6.8109 - acc: 0.5770 - val_loss: 7.0874 - val_acc: 0.5603
Epoch 3/20
3383/3383 [==============================] - 397s 117ms/step - loss: 6.7941 - acc: 0.5785 - val_loss: 7.0874 - val_acc: 0.5603
Epoch 4/20
3383/3383 [==============================] - 401s 119ms/step - loss: 6.7941 - acc: 0.5785 - val_loss: 7.0874 - val_acc: 0.5603
Epoch 5/20
3383/3383 [==============================] - 369s 109ms/step - loss: 6.7941 - acc: 0.5785 - val_loss: 7.0874 - val_acc: 0.5603
Epoch 6/20
3383/3383 [==============================] - 384s 113ms/step - loss: 6.7941 - acc: 0.5785 - val_loss: 7.0874 - val_acc: 0.5603

Epoch 00006: ReduceLROnPlateau reducing lear

In [ ]:
# save finished model
save_model(log_dir=log_dir, config=config, model=model)

In [ ]:
# save loss, accuracy, lr, and bs values across epochs as json
acc_loss_lr_bs = {'val_loss': val_loss,
                  'val_acc': val_acc,
                  'loss': loss,
                  'acc': acc,
                  'lr': [np.float64(i) for i in lr],
                  'bs': bs
                  }
acc_loss_lr_bs_path = os.path.join(log_dir, 'acc_loss_lr_bs.json')
with open(acc_loss_lr_bs_path, 'w') as f:
    json.dump(acc_loss_lr_bs, f, indent=4, sort_keys=True)

In [ ]:
# evaluate model
print('Calculating final score...')
x_data, y_data = retrieve_data(data_dict=data_dict, image_dir=test_dir)
score = model.evaluate(x_data, y_data, batch_size=batch_size)
print('Final score:', score)

In [ ]:
print('Completed program')

In [ ]:
# PLOTTING FOR TESTING
import matplotlib.pyplot as plt

In [ ]:
# load dict from json
d = None
with open(acc_loss_lr_bs_path, 'r') as f:
    d = json.load(f)

In [ ]:
# plot dict values
plt.figure(figsize=(20,10))

# summarize history for accuracy
plt.subplot(2, 2, 1)
plt.plot(d['acc'])
plt.plot(d['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

# summarize history for loss
plt.subplot(2, 2, 2)
plt.plot(d['loss'])
plt.plot(d['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

# summarize history for lr
plt.subplot(2, 2, 3)
plt.plot(d['lr'])
plt.title('model lr')
plt.ylabel('lr')
plt.xlabel('epoch')
plt.legend(['lr'], loc='upper left')

# summarize history for bs
plt.subplot(2, 2, 4)
plt.plot(d['bs'])
plt.title('model bs')
plt.ylabel('bs')
plt.xlabel('epoch')
plt.legend(['bs'], loc='upper left')

plt.show()